Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [ ]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [ ]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [ ]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [ ]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [ ]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [ ]:
import math
import numpy as np

def fitness(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.1}
    

def evolution(state: Nim) -> Nimply:
    """ (1,lambda) evolution strategy """
    lambda_ = 10
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    best_move , best_sum = None, math.inf
    for _ in range(lambda_):
        row, num_objects = random.choice(possible_moves)
        tmp = deepcopy(state)
        tmp.nimming(Nimply(row, num_objects))
        tmp_sum = fitness(tmp)
        if (best_sum > tmp_sum and best_sum != 0) or best_move is None:
            best_move, best_sum = (row, num_objects), tmp_sum
    return Nimply(*best_move)


In [ ]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        logging.debug(f"ply: {ply}")
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [ ]:
#logging.getLogger().setLevel(logging.INFO)

strategy = (gabriele, simulation_choice)
player_wins = [0, 0]
for i in range(100):
    nim = Nim(7)
    #logging.info(f"init : {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim)
        #logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        #logging.info(f"status: {nim}")
        player = 1 - player
    player_wins[player] += 1
print(player_wins)
#logging.info(f"status: Player {player} won!")


In [ ]:
def number_wins(state : Nim):
    strategy = (pure_random, pure_random)
    player_wins = [0, 0]
    for i in range(10):
        nim = deepcopy(state)
        #logging.info(f"init : {nim}")
        player = 0
        while nim:
            ply = strategy[player](nim)
            #logging.info(f"ply: player {player} plays {ply}")
            nim.nimming(ply)
            #logging.info(f"status: {nim}")
            player = 1 - player
        player_wins[player] += 1
    return player_wins[0]


In [ ]:
def simulation_choice(state : Nim) :
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    best_move , best_sum = None, math.inf
    for move in possible_moves:
        tmp = deepcopy(state)
        tmp.nimming(Nimply(*move))
        tmp_sum = number_wins(tmp)
        if (best_sum > tmp_sum and best_sum != 0) or best_move is None:
            best_move, best_sum = move, tmp_sum
    return Nimply(*best_move)

In [ ]:
nim = Nim(4)
while nim :
    possible_moves = [(r, o) for r, c in enumerate(nim.rows) for o in range(1, c + 1)]
    ply = simulation_choice(nim)
    nim.nimming(ply)
    print(len(possible_moves))
    input()

In [ ]:
class PlayerNim():
    def __init__(self,weights,moves) -> None:
        self.weights = weights
        self.rules = dict()
        self.moves = moves
        
        for i,move in enumerate(moves) :
            self.rules[move] = self.weights[i]

    def __repr__(self) -> str:
        
        return "PlayerNim : ( % s )" % self.weights

    def play(self,state : Nim) -> Nimply :
        possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
        best_move , best_sum = None, math.inf
        for move in possible_moves:
            tmp_sum = self.rules[move]
            if best_sum > tmp_sum or best_move is None:
                best_move, best_sum = move, tmp_sum
        return Nimply(*best_move)
    
    @staticmethod
    def crossover(player1,player2):
        weights = []
        for i in range(len(player1.weights)) :
            weights.append(random.choice([player1.weights[i],player2.weights[i]]))
        return PlayerNim(weights,player1.moves)
    
    @staticmethod
    def mutation(player,mutation_rate):
        weights = []
        for i in range(len(player.weights)) :
            if random.random() < mutation_rate :
                weights.append(random.random())
            else :
                weights.append(player.weights[i])
        return PlayerNim(weights,player.moves)

    def fitness(self):
        wins = [0,0]
        for _ in range(100):
            nim = Nim(4)
            i = 0
            while nim :
                if i % 2 == 0:
                    ply = self.play(nim)
                else :
                    ply = optimal(nim)
                nim.nimming(ply)
                i = 1 - i
            wins[i] += 1
        return wins[0]
        

class TrainingPlayerNim():
    def __init__(self, size : int) -> None:
        self.state = Nim(size)
        self.moves = [(r, o) for r, c in enumerate(self.state.rows) for o in range(1, c + 1)]
        self.population = []
        self.weights = []
        self.init_population(20)
        self.ration = 0.5

    def init_population(self, size : int) -> None:
        for _ in range(size):
            self.population.append(PlayerNim([random.random() for _ in range(len(self.moves))],self.moves))

    def generation(self) -> None:
        self.population.sort(key=lambda x : x.fitness() , reverse=True)
        self.population = self.population[:len(self.population)//2]
        for _ in range(len(self.population)):
            n = random.random()
            if n > self.ration :
                self.population.append(PlayerNim.crossover(random.choice(self.population),random.choice(self.population)))
            else:
                self.population.append(PlayerNim.mutation(random.choice(self.population),0.1))
        self.ration = 0.9 * self.ration

    def train(self) :
        for _ in range(40):
            self.generation()
            print(self.population[0].fitness())
        
        
            


In [ ]:
generation = TrainingPlayerNim(4)
generation.train()

In [ ]:
player = TrainingPlayerNim(4)
print(player.population)
player.population.sort(key=lambda x : x.fitness() , reverse=True)
player.population = player.population[:len(player.population)//2]
print(player.population)
        

In [ ]:
nim = Nim(4)
nim.rows
moves = [(r, o) for r, c in enumerate(nim.rows) for o in range(1, c + 1)]
move = random.choice(moves)
print(nim.rows)
print(move)
nim.nimming(Nimply(*move))
print(nim.rows)
en = enumerate(nim.rows)
print(list(en))

In [ ]:
# class PlayerNimColumn():
#     def __init__(self , weights : list, columns : int) -> None:
#         self.rules = dict()
#         self.moves = moves
        
#         for i in range(columns) :
#             self.rules[i] = weights[i]

#     def __repr__(self) -> str:
        
#         return "PlayerNim : ( % s )" % self.rules

#     def play(self,state : Nim) -> Nimply :
# #        possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
#         num_column = len(state.rows)
#         best_move , best_sum = None, math.inf
#         for move in possible_moves:
#             tmp_sum = self.rules[num_column]
#             if best_sum > tmp_sum or best_move is None:
#                 best_move, best_sum = move, tmp_sum
#         return Nimply(*best_move)
    
#     @staticmethod
#     def crossover(player1,player2):
#         weights = []
#         for i in range(len(player1.weights)) :
#             weights.append(random.choice([player1.weights[i],player2.weights[i]]))
#         return PlayerNim(weights,player1.moves)
    
#     @staticmethod
#     def mutation(player,mutation_rate):
#         weights = []
#         for i in range(len(player.weights)) :
#             if random.random() < mutation_rate :
#                 weights.append(random.random())
#             else :
#                 weights.append(player.weights[i])
#         return PlayerNim(weights,player.moves)

#     def fitness(self):
#         wins = [0,0]
#         for _ in range(100):
#             nim = Nim(4)
#             i = 0
#             while nim :
#                 if i % 2 == 0:
#                     ply = self.play(nim)
#                 else :
#                     ply = optimal(nim)
#                 nim.nimming(ply)
#                 i = 1 - i
#             wins[i] += 1
#         return wins[0]


In [ ]:
def modifica_primo_non_zero(tupla):
    # Converte la tupla in una lista
    lista_modificabile = list(tupla)

    # Trova il primo campo non-zero
    for i in range(len(lista_modificabile)):
        if lista_modificabile[i] != 0:
            # Modifica il primo campo non-zero
            lista_modificabile[i] = lista_modificabile[i] -1
            break

    # Converte la lista modificata di nuovo in una tupla
    tupla_modificata = tuple(lista_modificabile)

    return tupla_modificata

def generate_state(current_state : list,last_state : tuple):
    if (sum(last_state) > 0):
        current_state.append(last_state)
        tmp = modifica_primo_non_zero(last_state)
        generate_state(current_state,tmp)
    else :
        current_state.append(last_state)
    return current_state
    
state = generate_state([],Nim(4).rows)
print(state)

In [ ]:
def generate_state(current_state : list,last_state : list,pos : int):
    if (last_state[pos] > 0):
        current_state.append(last_state)
        tmp = deepcopy(last_state)
        tmp[pos] = tmp[pos] - 1
        generate_state(current_state,tmp,pos)
    else :
        current_state.append(last_state)
    return current_state


def generate_all_states(state : tuple) :
    state = list(state)
    all_states = []
    for i in range(len(state)):
        all_states.append(generate_state([],state,i))
    return all_states
        
print(generate_all_states(Nim(4).rows))

In [ ]:
from itertools import product

def generate_lists(numbers):
    n = len(numbers)
    # Genera tutte le possibili combinazioni di numeri con lunghezza n
    all_combinations = list(product(range(0, max(numbers) + 1), repeat=n))

    # Filtra le combinazioni per soddisfare il requisito
    valid_combinations = [list(combination) for combination in all_combinations if all(x <= y for x, y in zip(combination, numbers))]
    
    return valid_combinations[1:]

# Esempio di utilizzo con la lista [1, 3, 5, 7]
numbers = list(Nim(4).rows)
len(generate_lists(numbers))

In [ ]:
def random_moves(states : list) -> dict: 
    rules = dict()
    for state in states:
        rules[tuple(state)] = random.choice([(r, o) for r, c in enumerate(state) for o in range(1, c + 1)])

    return rules

states = generate_lists(list(Nim(4).rows)) 
rules = random_moves(states)
rules

In [91]:
class PlayerNimStates():
    def __init__(self,rules : dict) -> None:
        self.rules = rules
        self.fitness = None

    def play(self,state : Nim) -> Nimply:
        return self.rules[state.rows]

    @staticmethod
    def crossover(player1,player2):
        rules = dict()
        for state in player.rules.keys():
            rules[state] = random.choice([player1.rules[state],player2.rules[state]])
        return PlayerNimStates(rules)
    
    @staticmethod
    def mutation(player,mutation_rate):
        rules = dict()
        for state in player.rules.keys():
            if random.random() < mutation_rate :
                rules[state] = random.choice([(r, o) for r, c in enumerate(state) for o in range(1, c + 1)])
            else :
                rules[state] = player.rules[state]
        return PlayerNimStates(rules)

    def compute_fitness(self):
        wins = [0,0]
        for _ in range(100):
            nim = Nim(4)
            i = 0
            while nim :
                if i % 2 == 0:
                    ply = self.play(nim)
                else :
                    ply = optimal(nim)
                nim.nimming(ply)
                i = 1 - i
            wins[i] += 1
        return wins[0]
    
    def eval(self):
        wins = [0,0]
        for _ in range(100):
            nim = Nim(4)
            i = 0
            while nim :
                if i % 2 == 0:
                    ply = self.play(nim)
                else :
                    ply = optimal(nim)
                nim.nimming(ply)
                i = 1 - i
            wins[i] += 1
        return wins[0]
    

class TrainingPlayerNimStates():
    def __init__(self, size : int , pop_size : int) -> None:
        self.state = Nim(size)
        self.population = []
        self.weights = []
        self.ration = 0.8
        self.possible_states = generate_lists(list(nim.rows))
        self.init_population(pop_size)

    def init_population(self, size : int) -> None:
        for _ in range(size):
            temp = PlayerNimStates(random_moves(self.possible_states))
            temp.fitness = temp.compute_fitness()
            self.population.append(temp)

    def generation(self) -> None:
        for _ in range(len(self.population)):
            n = random.random()
            if n > self.ration :
                temp = PlayerNimStates.crossover(random.choice(self.population),random.choice(self.population))
            else:
                temp = PlayerNimStates.mutation(random.choice(self.population),0.1)
            temp.fitness = temp.compute_fitness()
            self.population.append(temp)    
        self.population.sort(key=lambda x : x.fitness , reverse=True)
        self.population = self.population[:len(self.population)//2]
        self.ration = 0.9 * self.ration

    def train(self,epoch = 10):
        for _ in range(epoch):
            self.generation()
            print(self.population[0].eval() , self.population[0].fitness)
        return self.population[0]

#nim = Nim(4)
#possible_states = generate_lists(list(nim.rows))
#player = PlayerNimStates(random_moves(possible_states))
#player.fitness()

player = TrainingPlayerNimStates(4,40).train(40)

39 52
41 53
53 54
62 59
59 59
65 67
74 67
79 74
71 74
76 74
71 76
63 76
67 76
85 82
78 82
72 82
80 90
81 90
79 90
84 90
88 91
86 91
92 91
86 94
90 95
87 95
88 95
92 95
82 95
91 95
84 95
90 95
97 98
93 98
97 98
96 98
96 98
89 98
94 98
94 98


In [105]:
player.eval()

92

In [113]:
wins = [0,0]
actual_player = deepcopy(player)
for _ in range(100):
            nim = Nim(4)
            i = 0
            while nim :
                if i == 0:
                    ply = actual_player.play(nim)
                else :
                    ply = optimal(nim)
                nim.nimming(ply)
                i = 1 - i
            wins[i] += 1
print(wins)

[93, 7]


In [114]:
print(player.rules)

{(0, 0, 0, 1): (3, 1), (0, 0, 0, 2): (3, 2), (0, 0, 0, 3): (3, 1), (0, 0, 0, 4): (3, 3), (0, 0, 0, 5): (3, 1), (0, 0, 0, 6): (3, 4), (0, 0, 0, 7): (3, 1), (0, 0, 1, 0): (2, 1), (0, 0, 1, 1): (3, 1), (0, 0, 1, 2): (3, 2), (0, 0, 1, 3): (3, 2), (0, 0, 1, 4): (3, 2), (0, 0, 1, 5): (3, 2), (0, 0, 1, 6): (3, 2), (0, 0, 1, 7): (3, 1), (0, 0, 2, 0): (2, 1), (0, 0, 2, 1): (2, 2), (0, 0, 2, 2): (2, 1), (0, 0, 2, 3): (2, 1), (0, 0, 2, 4): (3, 3), (0, 0, 2, 5): (2, 2), (0, 0, 2, 6): (3, 2), (0, 0, 2, 7): (3, 3), (0, 0, 3, 0): (2, 2), (0, 0, 3, 1): (3, 1), (0, 0, 3, 2): (2, 3), (0, 0, 3, 3): (3, 3), (0, 0, 3, 4): (2, 3), (0, 0, 3, 5): (3, 2), (0, 0, 3, 6): (3, 5), (0, 0, 3, 7): (2, 3), (0, 0, 4, 0): (2, 3), (0, 0, 4, 1): (2, 2), (0, 0, 4, 2): (2, 2), (0, 0, 4, 3): (2, 4), (0, 0, 4, 4): (2, 2), (0, 0, 4, 5): (3, 3), (0, 0, 4, 6): (2, 3), (0, 0, 4, 7): (3, 1), (0, 0, 5, 0): (2, 4), (0, 0, 5, 1): (2, 2), (0, 0, 5, 2): (3, 2), (0, 0, 5, 3): (3, 1), (0, 0, 5, 4): (2, 4), (0, 0, 5, 5): (2, 5), (0, 0, 5,